In [1]:
import sqlite3
import yaml

In [2]:
fp = r"E:\WBH\Games\eve forever\EVE Data\Hhoboleaks Export\cloneStates.yaml"
with open(fp, 'r', encoding='utf-8') as f:
    omega_list = yaml.load(f, yaml.CLoader)
fp = r"E:\WBH\Games\eve forever\EVE Data\sde\sde\fsd\typeDogma.yaml"
with open(fp, 'r', encoding='utf-8') as f:
    dogma_data = yaml.load(f, yaml.CLoader)

In [6]:
db = sqlite3.connect('./skills.db')
cursor = db.cursor()

In [8]:
_sql = """create table if not exists skill_constant (
    skill_id integer primary key not null,
    time_factor integer not null default 0,
    alpha_level integer not null default 0
);"""
cursor.execute(_sql)
db.commit()

In [9]:
omega_map = {}
for value in omega_list.values():
    if 'skills' in value.keys():
        skills: dict = value['skills']
        for skill_id, skill_value in skills.items():
            if skill_id not in omega_map.keys() or omega_map[skill_id] < skill_value:
                omega_map[skill_id] = skill_value

In [26]:
skill_map = {}
for key, value in dogma_data.items():
    factor = 0
    if 'dogmaAttributes' in value.keys():
        for attr in value['dogmaAttributes']:
            if int(attr['attributeID']) == 275:
                factor = int(attr['value'])
                break
    omega = 0 if key not in omega_map.keys() else omega_map[key]
    skill_map[key] = {"factor": factor, "omega": omega}

In [33]:
_sql = "insert into skill_constant values (%d, %d, %d)"
for skill_id, data in skill_map.items():
    cursor.execute(_sql % (skill_id, data['factor'], data['omega']))
db.commit()

In [34]:
_sql = "delete from skill_constant where time_factor == 0;"
cursor.execute(_sql)
db.commit()

In [35]:
db.close()